In [1]:
# | default_exp backend.llms.client

In [1]:
# | export
import os
from typing import Dict, Any, List, Optional, Iterator
from openai import OpenAI
from rich.console import Console
from rich.markdown import Markdown

In [2]:
# | export
from agentic.configs.loader import get_model_config, get_settings_config
from agentic.llms.response_processor import ResponseProcessor
from agentic.llms.streaming_handler import StreamingHandler

In [3]:
# | export
class LLMClient:
    """Enhanced LLM client"""
    
    def __init__(self, model: Optional[str] = None, base_url: Optional[str] = None, 
                 api_key: Optional[str] = None):
        # Load config defaults
        model_config = get_model_config()
        settings_config = get_settings_config()
        
        # Set parameters with fallbacks
        self.model = model or model_config.get('name', 'qwen3:8b')
        self.base_url = base_url or model_config.get('url', 'http://localhost:11434/v1')
        self.api_key = api_key or model_config.get('api_key', 'ollama')

        
        # Initialize OpenAI client
        self.client = OpenAI(base_url=self.base_url, 
                             api_key=self.api_key)
        
        # Initialize processors
        self.response_processor = ResponseProcessor()
        self.streaming_handler = StreamingHandler()
        
        # Validate connection
        self._validate_connection()
    
    def _validate_connection(self):
        """Validate LLM connection"""
        try:
            # Test with a simple request
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": "test"}],
                max_tokens=1,
                timeout=5
            )
            return True
        except Exception as e:
            print(f"⚠️ LLM connection warning: {e}")
            return False
    
    def create_completion(self, messages: List[Dict[str, Any]], 
                         tools: Optional[List[Dict]] = None,
                         stream: bool = True, **kwargs) -> Any:
        """Create chat completion with optional tools"""
        completion_params = {
            "model": self.model,
            "messages": messages,
            "stream": stream,
            **kwargs
        }
        
        if tools:
            completion_params["tools"] = tools
            completion_params["tool_choice"] = "auto"
        
        try:
            return self.client.chat.completions.create(**completion_params)
        except Exception as e:
            raise RuntimeError(f"LLM completion failed: {e}")
    
    def process_response(self, response: Any, console: Optional[Console] = None) -> Dict[str, Any]:
        """Process non-streaming response"""
        return self.response_processor.process_response(response, console)
    
    def handle_streaming_response(self, response: Iterator, console: Optional[Console] = None) -> Dict[str, Any]:
        """Handle streaming response"""
        return self.streaming_handler.handle_streaming_response(response, console)
    
    def get_model_info(self) -> Dict[str, Any]:
        """Get information about the current model"""
        return {
            "model": self.model,
            "base_url": self.base_url,
            "api_key_set": bool(self.api_key),
            "connection_valid": self._validate_connection()
        }


In [4]:
# | hide
# Example usage 

system_prompt = {
    "role": "system",
    "content": (
        "You are a helpful assistant. "
        "Return all your responses using valid **Markdown syntax**, including:\n"
        "- Headers (`#`, `##`)\n"
        "- Bullet points\n"
        "- Code blocks (triple backticks)\n"
        "- Bold / italic text\n"
        "- Quotes and tables if needed\n\n"
        "Also, try to use formatting that works well with color rendering in terminals using `rich`."
    )
}



from rich.console import Console
client = LLMClient()
response = client.create_completion(
    messages=[
    system_prompt,
    {"role": "user", "content": "Explain how a neural network works."}
    ],
    tools=[],
    stream=False
)
# result = client.handle_streaming_response(response,Console() )
result = client.process_response(response,Console())


╭─────────────────────── 🤔 Thinking ───────────────────────╮
Okay, I need to explain how a neural network works. Let me start by recalling what I know. Neural networks are inspired by the human brain, right? They consist of layers of nodes, like an input layer, hidden layers, and an output layer. Each node is connected to the next, and these connections have weights.

First, I should break down the components. The input layer receives the data. Then, each hidden layer processes the data through weights and biases, applying an activation function. The output layer gives the final result. But how do they actually process the data?

Maybe I should explain the forward propagation step. The input data is multiplied by the weights, summed with biases, and passed through an activation function. This process happens layer by layer. The activation function introduces non-linearity, which is crucial because without it, the network can't learn complex patterns.

Wait, but what are the activation

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                           How a Neural Network Works                                            ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

A neural network is a computational model inspired by the human brain, designed to recognize patterns and make     
predictions. Here's a breakdown of its core mechanics:                                                             

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                1. Core Components                                                 

 • Input Layer: Receives raw data (e.g., pixel values for images).                                                 
 • Hidden Layers: Intermediate layers that process data through weighted connections.                              
 • Output Layer: Produces the final result (e.g., class labels or numerical predictions).                          
 • Weights & Biases: Parameters that determine the strength of connections between nodes.                          
 • Activation Functions: Introduce non-linearity to enable complex pattern recognition.                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                              2. Forward Propagation                                               

Data flows through the network in stages:                                                                          

 1 Input → Hidden Layer:                                                                                           
    • Each node computes:                                                                                          
      $$ z = (W \cdot x) + b $$                                                                                    
      where $ W $ = weights, $ x $ = input, $ b $ = bias.                                                          
    • Apply an activation function:                                                                                
      $$ a = f(z) $$                                                                                               
      (e.g., ReLU, sigmoid, or tanh).                                                                              
 2 Hidden → Output Layer:                                                                                          
    • Repeat the same process to compute the final output.                                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                3. Training Process                                                

The network learns by minimizing errors through backpropagation:                                                   

 1 Loss Function:                                                                                                  
    • Measures prediction error (e.g., MSE for regression, cross-entropy for classification).                      
 2 Gradient Descent:                                                                                               
    • Calculates gradients of the loss with respect to weights using the chain rule.                               
    • Updates weights to reduce loss:                                                                              
      $$ W_{\text{new}} = W_{\text{old}} - \eta \cdot \frac{\partial L}{\partial W} $$                             
      where $ \eta $ = learning rate.                                                                              

───────────────────────────────────────────